# Xây dựng mô hình Naïve ngây thơ trên tập dữ liệu hành vi của khách hàng

### PHẦN 1 — Import thư viện và nạp dữ liệu vào Notebook

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Đọc dữ liệu từ file
data = pd.read_csv('Customer_Behaviour.csv')

# Hiển thị 5 dòng đầu tiên
data.head()


,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


Import các thư viện cơ bản như numpy, pandas để xử lý dữ liệu và sklearn để xây dựng mô hình.

Đọc file Customer_Behaviour.csv chứa thông tin hành vi khách hàng.

Kiểm tra 5 dòng đầu tiên để hiểu cấu trúc và kiểu dữ liệu (các cột như Age, Gender, Annual Income, Spending Score...).

Mục tiêu là dự đoán khách hàng mua (1) hoặc không mua (0) sản phẩm dựa trên các đặc trưng hành vi.

### PHẦN 2 — Xử lý dữ liệu trước khi xây dựng mô hình

In [3]:

# Kiểm tra thông tin tổng quát của tập dữ liệu
print("Thông tin dữ liệu:")
print(data.info())

# Kiểm tra giá trị thiếu
print("\nSố lượng giá trị thiếu:")
print(data.isnull().sum())

# Nếu có giá trị thiếu, loại bỏ các hàng đó
data = data.dropna()

# Nếu có cột chuỗi (object), mã hóa thành số
for col in data.select_dtypes(include=['object']).columns:
    data[col] = data[col].astype('category').cat.codes

# Chia dữ liệu thành X (đặc trưng) và y (nhãn)
X = data.drop(columns=['Purchased'])  # giả sử cột 'Purchased' là nhãn
y = data['Purchased']

# Chia thành tập huấn luyện (80%) và kiểm tra (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Thông tin dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB
None

Số lượng giá trị thiếu:
User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64


Kiểm tra thông tin dữ liệu (.info()) để xem các cột, kiểu dữ liệu và có giá trị trống hay không.

Nếu có giá trị NaN → loại bỏ hoặc thay thế (ở đây chọn dropna() vì dữ liệu nhỏ).

Mã hóa các cột dạng chuỗi (ví dụ: Gender, Profession, Type) thành số để Naïve Bayes hiểu được.

Chia dữ liệu thành hai phần:

X: đặc trưng hành vi (tuổi, thu nhập, nghề nghiệp, v.v.)

y: nhãn mục tiêu (ví dụ: cột Purchase)

Sau đó chia thành 80% train và 20% test để kiểm định mô hình.

### PHẦN 3 — Xây dựng mô hình Naïve Bayes

In [4]:

# Khởi tạo mô hình Gaussian Naive Bayes
model = GaussianNB()

# Huấn luyện mô hình
model.fit(X_train, y_train)


GaussianNB()

Sử dụng GaussianNB() vì các đặc trưng trong tập dữ liệu là dạng số thực liên tục (tuổi, thu nhập, chi tiêu,...).

Huấn luyện mô hình bằng model.fit(X_train, y_train).

Quá trình này giúp mô hình học xác suất điều kiện giữa từng đặc trưng và nhãn mua hàng.

### PHẦN 4 — Dự đoán và đánh giá mô hình

In [5]:
# Dự đoán trên tập test
y_pred = model.predict(X_test)

# Tính các chỉ số đánh giá
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"🎯 Độ chính xác (Accuracy): {accuracy:.2f}")
print("\n🧮 Ma trận nhầm lẫn (Confusion Matrix):")
print(conf_matrix)
print("\n📋 Báo cáo phân loại (Classification Report):")
print(classification_rep)


🎯 Độ chính xác (Accuracy): 0.93

🧮 Ma trận nhầm lẫn (Confusion Matrix):
[[51  1]
 [ 5 23]]

📋 Báo cáo phân loại (Classification Report):
              precision    recall  f1-score   support

           0       0.91      0.98      0.94        52
           1       0.96      0.82      0.88        28

    accuracy                           0.93        80
   macro avg       0.93      0.90      0.91        80
weighted avg       0.93      0.93      0.92        80



Dự đoán hành vi mua hàng trên tập test bằng predict().

Tính accuracy (độ chính xác), confusion matrix (ma trận nhầm lẫn), và classification report (precision, recall, F1).

Ma trận nhầm lẫn giúp ta biết mô hình phân loại sai bao nhiêu mẫu.

Báo cáo phân loại cho biết mức độ chính xác từng lớp — ví dụ: mô hình dự đoán chính xác khách hàng mua cao hơn khách không mua.

Nếu accuracy > 0.8 → mô hình có khả năng dự đoán hành vi khá tốt.

### PHẦN 5 — Kiểm tra mô hình trên vài mẫu cụ thể

In [7]:
# Lấy 10 mẫu ngẫu nhiên từ tập test để xem dự đoán
sample = X_test.sample(10, random_state=1)

# Thực hiện dự đoán
pred_sample = model.predict(sample)

# Xác suất dự đoán cho từng lớp
proba_sample = model.predict_proba(sample)

# Gộp tất cả lại để so sánh dễ hơn
compare_df = sample.copy()
compare_df['Thực tế'] = y_test.loc[sample.index].values
compare_df['Dự đoán'] = pred_sample
compare_df['Xác suất lớp 0'] = proba_sample[:, 0]
compare_df['Xác suất lớp 1'] = proba_sample[:, 1]

# Hiển thị kết quả
display(compare_df)


,User ID,Gender,Age,EstimatedSalary,Thực tế,Dự đoán,Xác suất lớp 0,Xác suất lớp 1
78,15667265,0,28,87000,0,0,0.916098,0.083902
225,15622171,1,37,53000,0,0,0.878495,0.121505
193,15662901,1,19,70000,0,0,0.984570,0.015430
73,15782530,0,33,113000,0,1,0.488129,0.511871
148,15779862,1,29,61000,0,0,0.945454,0.054546
181,15774727,0,31,71000,0,0,0.913226,0.086774
70,15756932,1,25,80000,0,0,0.949980,0.050020
388,15672330,1,47,34000,1,0,0.538857,0.461143
141,15666141,0,18,68000,0,0,0.986774,0.013226
45,15639277,1,23,20000,0,0,0.974352,0.025648


Mục tiêu: Kiểm tra mô hình hoạt động thực tế như thế nào trên một vài khách hàng cụ thể.

Lấy ngẫu nhiên 10 dòng từ tập kiểm tra, sau đó dự đoán bằng mô hình Naïve Bayes đã huấn luyện.

Sử dụng predict_proba() để xem xác suất mô hình tin tưởng vào mỗi lớp (ví dụ: “Không mua” = 0, “Mua” = 1).

Kết quả được gộp lại trong một bảng gồm:
Các đặc trưng gốc (ví dụ: tuổi, thu nhập, giới tính, v.v.)
Cột Thực tế (giá trị thật trong dữ liệu)
Cột Dự đoán (kết quả mô hình)
Hai cột Xác suất cho từng lớp giúp giải thích độ tin cậy.

Mô hình Naïve Bayes dự đoán khá ổn định: phần lớn các mẫu có cột “Dự đoán” trùng với “Thực tế”, chứng tỏ mô hình học được mối quan hệ giữa các đặc trưng và hành vi mua hàng.

Ở các dòng mà dự đoán sai, ta thấy xác suất hai lớp (0 và 1) khá cân bằng, nghĩa là mô hình không quá tự tin — đây là điều bình thường với dữ liệu có đặc trưng chồng lấn.

Những mẫu có xác suất lớp đúng > 0.8 cho thấy mô hình nhận diện khá chắc chắn, còn các mẫu có xác suất ~0.5 là các trường hợp biên khó phân biệt.

Tổng thể, kết quả chứng minh Naïve Bayes hoạt động tốt với dữ liệu khách hàng có phân bố gần độc lập giữa các đặc trưng, tuy nhiên có thể cải thiện hơn nếu chuẩn hoá dữ liệu hoặc bổ sung đặc trưng hành vi cụ thể hơn.